## Development Notebook

Used for trying out widget code changes

In [1]:
# These settings automatically re-render the widget when the JavaScipt file has changed

%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, IntSlider, FloatSlider, Box, jslink, HTML, DOMWidget

In [3]:
import json

with open('small_styled_crustal.geojson') as json_file:
    styled_crustal = json.load(json_file)
    json_file.close()

polygon = {"type":"FeatureCollection","features": [{
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [[
            [174.934,-41.188],
            [174.925,-41.188],
            [174.925,-41.197],
            [174.934,-41.197],
            [174.934,-41.188]
            ]],
        "type": "Polygon"
    }
}]}

color_map = {0.001: '#fcffa4',
 0.0005: '#fbbe23',
 0.0002: '#ea632a',
 0.0001: '#bc3754',
 5e-05: '#7f1e6c',
 2e-05: '#2b0b57',
 1e-05: '#000004'}

In [4]:
from nzshm_rupture_widget import CesiumWidget, SliderWidget, FullScreenWidget, HomeWidget, legend, selection_slider, MapLayoutBuilder, transparency_button, ValueButtonWidget, geojson_3d_map, decorate

In [5]:
cesium = CesiumWidget(data = [styled_crustal, polygon])

html = HTML("<b>Section Detail</b><br/><p>hover over fault sections for more details.</p>", style={"background":"white", "border-radius":"5px"})
html.add_class("mapHtmlWidget")
def on_hover(msg):
    properties = msg["properties"]
    value = f"<b>{properties['FaultName']}</b>"
    value += "<br />"
    value += f"Dip: {properties['DipDeg']}</br>"
    value += f"Rake: {properties['Rake']}</br>"
    value += f"Lower depth: {round(properties['LowDepth'],3)}</br>"    
    value += f"Participation rate: {properties['annual_rate']:.2E}</br>" 
    html.value = value

cesium.on_hover(on_hover)
cesium.hover_style = {"fillColor":"red", "fillOpacity":"0.6", "color":"green"}
cesium.no_info = True
cesium.globe_opacity = 1

alpha_slider = FloatSlider(min=0, max=1, value=0.5)
jslink((cesium, "globe_opacity"), (alpha_slider, "value"))

MapLayoutBuilder(cesium)\
    .bottom_right(
        html, 
        selection_slider(cesium))\
    .top_left(
        HomeWidget(cesium),
        transparency_button(cesium, [1, 0.5, 0.2, 0]),
        FullScreenWidget(),
        legend("Rupture Rate/yr", color_map))\
    .top_right(
        alpha_slider)\
    .build()

GridBox(children=(CesiumWidget(data=[{'type': 'FeatureCollection', 'features': [{'id': '216', 'type': 'Feature…